This code will iterate through the Heritage Indicator workbooks (saved in the "source_folder"), copy the workbooks and add a new column to the longitudinal data tables for the following year, ignoring any non-longitudinal or uniquely structured tables (metadata on these tables is in the "lookup_table_path").

The variables and file paths neccessary for this code to work are described here:

lookup_table_path = the file path to the "HI Lookup Table and Metadata" Excel file you have downloaded for this years data collection process.

source_folder = the file path to the new folder you have created, containing all of last years Heritage Indicator workbooks (in Excel format).

destination_folder = the file path to the location you want to save the newly amended workbooks to.

In [ ]:
import os
import re
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from copy import copy
from openpyxl.utils.dataframe import dataframe_to_rows

# Paths (adjust these paths to wherever you have saved/created the files/folders)
lookup_table_path = r'C:\filepath\HI Lookup Table and Metadata.xlsx'
source_folder = r'C:\filepath\Last Years HI Files'
destination_folder = r'C:\filepath\Updated HI Files'

# Function to determine the next logical year for the new column
def get_next_year_column_name(last_column_name):
    financial_year_pattern = re.compile(r'(\d{4})/(\d{2})')
    single_year_pattern = re.compile(r'(\d{4})')
    
    match = financial_year_pattern.match(last_column_name) or single_year_pattern.match(last_column_name)
    if match:
        year_str = match.group(0)
        if financial_year_pattern.match(year_str):
            start_year = int(year_str[:4])
            next_start_year = start_year + 1
            next_end_year = (start_year + 2) % 100
            return f"{next_start_year}/{next_end_year:02d}"
        elif single_year_pattern.match(year_str):
            year = int(year_str)
            return str(year + 1)
    else:
        raise ValueError(f"Unrecognized year format in column name: {last_column_name}")

def copy_cell_attributes(source_cell, target_cell):
    target_cell.value = source_cell.value
    
    if source_cell.has_style:
        target_cell.font = copy(source_cell.font)
        target_cell.border = copy(source_cell.border)
        target_cell.fill = copy(source_cell.fill)
        target_cell.number_format = copy(source_cell.number_format)
        target_cell.protection = copy(source_cell.protection)
        target_cell.alignment = copy(source_cell.alignment)

def recreate_table(new_sheet, original_table, data, start_row, start_col_index):
    # Create a new table with the same range but updated with potentially new columns
    end_col_index = start_col_index + len(data.columns) - 1
    end_row = start_row + len(data)  # This ensures the table does not extend beyond the data
    end_col_letter = new_sheet.cell(row=start_row, column=end_col_index).coordinate.split('1')[0]
    
    new_table = Table(displayName=original_table.displayName, ref=f"{new_sheet.cell(row=start_row, column=start_col_index).coordinate}:{end_col_letter}{end_row}")
    
    # Copy table style
    if original_table.tableStyleInfo:
        new_style = TableStyleInfo(
            name=original_table.tableStyleInfo.name,
            showFirstColumn=original_table.tableStyleInfo.showFirstColumn,
            showLastColumn=original_table.tableStyleInfo.showLastColumn,
            showRowStripes=original_table.tableStyleInfo.showRowStripes,
            showColumnStripes=original_table.tableStyleInfo.showColumnStripes
        )
        new_table.tableStyleInfo = new_style
    
    new_sheet.add_table(new_table)

def process_workbook(file_path, output_path, table_names):
    workbook = load_workbook(file_path)
    new_wb = Workbook()
    new_wb.remove(new_wb.active)  # Remove default sheet
    
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        new_sheet = new_wb.create_sheet(title=sheet_name)

        # Copy all cells
        for row_idx, row in enumerate(sheet.iter_rows(), start=1):
            for col_idx, cell in enumerate(row, start=1):
                new_cell = new_sheet.cell(row=row_idx, column=col_idx)
                copy_cell_attributes(cell, new_cell)
        
        if sheet_name in table_names:
            for table in sheet.tables.values():
                try:
                    start_cell, end_cell = table.ref.split(':')
                    start_col, start_row = coordinate_from_string(start_cell)
                    end_col, end_row = coordinate_from_string(end_cell)
                    start_col_index = column_index_from_string(start_col)
                    end_col_index = column_index_from_string(end_col)
                    
                    data = sheet.iter_rows(
                        min_row=start_row, max_row=end_row,
                        min_col=start_col_index, max_col=end_col_index,
                        values_only=True)
                    df = pd.DataFrame(data)
                    df.columns = df.iloc[0]
                    df = df.drop(0).reset_index(drop=True)
                    
                    last_column_name = df.columns[-1]
                    new_column_name = get_next_year_column_name(last_column_name)
                    df[new_column_name] = ''
                    
                    for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start=start_row):
                        for c_idx, value in enumerate(row, start=start_col_index):
                            new_sheet.cell(row=r_idx, column=c_idx, value=value)
                    
                    recreate_table(new_sheet, table, df, start_row, start_col_index)
                
                except ValueError as e:
                    print(f"Error processing table '{table.name}' in sheet '{sheet_name}' of file '{file_path}': {e}")
                    continue
        else:
            # Recreate original tables if no modification is needed
            for table in sheet.tables.values():
                start_cell, end_cell = table.ref.split(':')
                start_col, start_row = coordinate_from_string(start_cell)
                end_col, end_row = coordinate_from_string(end_cell)
                start_col_index = column_index_from_string(start_col)
                end_col_index = column_index_from_string(end_col)
                
                data = sheet.iter_rows(
                    min_row=start_row, max_row=end_row,
                    min_col=start_col_index, max_col=end_col_index,
                    values_only=True)
                df = pd.DataFrame(data)
                df.columns = df.iloc[0]
                df = df.drop(0).reset_index(drop=True)
                
                for r_idx, row in enumerate(dataframe_to_rows(df, index=False, header=True), start=start_row):
                    for c_idx, value in enumerate(row, start=start_col_index):
                        new_sheet.cell(row=r_idx, column=c_idx, value=value)
                
                recreate_table(new_sheet, table, df, start_row, start_col_index)
    
    new_wb.save(output_path)

# Read the lookup table
lookup_df = pd.read_excel(lookup_table_path)

filtered_lookup = lookup_df[
    (lookup_df['Longitudinal Dataset? Y/N/Y1'] == 'Y') &
    (lookup_df['Data Source Type'] != 'Discontinued')
]

os.makedirs(destination_folder, exist_ok=True)

for file_name in os.listdir(source_folder):
    if file_name.endswith('.xlsx'):
        file_path = os.path.join(source_folder, file_name)
        output_file = os.path.join(destination_folder, file_name)
        process_workbook(file_path, output_file, filtered_lookup['Sheet Name'].tolist())

print("Processing complete. Files saved to:", destination_folder)


As mentioned in the "Heritage Indicators Data Collection Documentation", as this process will amend any longitudinal data tables, but ignore any non-longitudinal or uniquely structured tables, you will be required to manually amend tables with new columns or rows which this code hasn't amended (unless the table is "Discontinued").

The information which tells you which tables need to be manually amended will be in the "HI Lookup Table and Metadata" Excel file.

See "Longitudinal Dataset? Y/Y1/N" in the "HI Lookup Table and Metadata Definitions" for more explanation.

